In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from net import Net

In [9]:
# Set variables to train the model.
no_cuda = True
seed = 1

use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

In [10]:
#Prepare Data Loader for Training and Validation
batch_size = 64
test_batch_size = 1000

mnist_transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.1307,), (0.3081,))
])

train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./dataset', train=True, download=True, transform=mnist_transform), 
        batch_size = batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('./dataset', train=False, download=True, transform=mnist_transform), 
        batch_size=test_batch_size, shuffle=True, **kwargs)

In [11]:
lr = 0.01
momentum = 0.5
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
cross_entropy_loss = nn.CrossEntropyLoss()

In [12]:

#Define Train function and Test function to validate.
def train(log_interval, model, device, train_loader, optimizer, epoch):

    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = cross_entropy_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(log_interval, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += cross_entropy_loss(output, target).item() 
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format
          (test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [15]:
log_interval = 200
epoch = 1
PATH = "./model/model.pt"
train(log_interval, model, device, train_loader, optimizer, epoch)
test(log_interval, model, device, test_loader)
torch.save(model, PATH)

Train Epoch: 1 [0/60000 (0%)]	Loss: 0.192569
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.283559
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.236447
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.066482
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.198892

Test set: Average loss: 0.0002, Accuracy: 9497/10000 (95%)



In [7]:
model = torch.load(PATH)
model.eval()

Net(
  (fc1): Linear(in_features=784, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)